In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
class mat_samp_full():
    
    def __init__(self,N,d,σ,dist='Gaussian'):
    
        M = int(np.floor(N/d))
        if dist=='Rademacher':
            self.X = (1-2*np.random.randint(2,size=M*N).reshape(N,M))/np.sqrt(M)
        elif dist=='Gaussian':
            self.X = np.random.randn(N,M)/np.sqrt(M)
        N2 = int(np.floor(N/2))
        self.X[:N2,:] *= np.sqrt(σ)
        self.shape = (N,N)
        self.dtype = np.double
    
    def __matmul__(self,v):
        
        return self.X@(self.X.T@v)

In [4]:
def support_gen(N,d,σ): # newtons' method to find all roots of df
    
    f = lambda x: -1/x + d/2*1/(x + 1) + d/2*1/(x + 1/σ)
    df = lambda x: 1/x**2 - d/2*1/(x + 1)**2 - d/2*1/(x + 1/σ)**2
    ddf = lambda x: -2/x**3 + d/(x + 1)**3 + d/(x + 1/σ)**3

    x0 = np.arange(-4,4,.005)
    for i in range(400):
        x0 = x0 - df(x0)/ddf(x0)
    x0 = f(x0[np.abs(x0)<100])
    
    return np.reshape(x0[np.unique(x0.round(decimals=10),return_index=True)[1]],(2,2))

In [5]:
def mu_U(x,a,b): 
    with np.errstate(invalid='ignore'):
        return (8/(np.pi*(b-a)**2))*np.sqrt((x-a)*(b-x))
    
def mu_T(x,a,b): 
    with np.errstate(divide='ignore',invalid='ignore'):
        return 1/(np.pi)/(np.sqrt((x-a)*(b-x)))

In [6]:
N = 5000
d = 0.3;
σ = 8.0

intervals_raw = support_gen(N,d,σ)
intervals = intervals_raw + np.array([[-1,1],[-1,1]]) * 0.001

a = intervals[0,0]
b = intervals[1,1]

weights = [.5,.5]


In [7]:
mu = lambda x,a,b: mu_U(x,a,b)

def mu2(x):
    mu_x = np.zeros_like(x)
    l = len(weights)
    for i in range(l):
        mask = (intervals[i,0]<x)*(x<intervals[i,1])
        mu_x[mask] += weights[i]*mu(x[mask],intervals[i,0],intervals[i,1])
    return mu_x

In [8]:
k = 30
s = 2*k

αU = np.ones(s+1)*(a+b)/2
βU = np.ones(s+1)*(b-a)/4

αU_unscaled = np.zeros(s+1)
βU_unscaled = np.ones(s+1)/2

α2,β2 = get_op_recurrence(intervals,weights,αU_unscaled,βU_unscaled,s)

In [9]:
np.random.seed(0)

n_trials = 10

α  = np.zeros((n_trials,k))
β  = np.zeros((n_trials,k))
mU = np.zeros((n_trials,k+1))
m2 = np.zeros((n_trials,k+1))

for trial_idx in range(n_trials):
    
    clear_output(wait=True)
    print(trial_idx)
    
    A = mat_samp_full(N,d,σ)
    v = np.sign(np.random.rand(N)-.5) / np.sqrt(N)

    
    Q,(α[trial_idx],β[trial_idx]) = lanczos_reorth(A,v,k)
    mU[trial_idx] = get_moments_from_lanczos(A,v,k,αU,βU,T=(α[trial_idx],β[trial_idx]))
    m2[trial_idx] = get_moments_from_lanczos(A,v,k,α2,β2,T=(α[trial_idx],β[trial_idx]))

9


In [10]:
xx = np.linspace(a,b,5000)

fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax = fig.add_axes([left, bottom, width, height])


AQ_U = eval_poly(xx,np.mean(mU,axis=0),αU,βU)
AQ_2 = eval_poly(xx,np.mean(m2,axis=0),α2,β2)

ax.plot(xx,AQ_U*mu_U(xx,a,b),**line_styles_nomarker['l3'])
ax.plot(xx,AQ_U*mu_U(xx,a,b),**line_styles_nomarker1['l3'])
ax.plot(xx,AQ_2*mu2(xx),**line_styles_nomarker['l1'])
    
for l in np.reshape(intervals_raw,-1):
    ax.axvline(l,color='k',lw=.5)

ax.grid(True,linestyle=':',linewidth=.5)

ax.set_ylim(0,.6)
ax.set_xlim(-1,17)

axins = inset_axes(ax, "100%","100%", loc='upper left', bbox_to_anchor=(0.35,.35,.5,.6), bbox_transform=ax.transAxes)
axins.plot(xx,AQ_U*mu_U(xx,a,b),**line_styles_nomarker['l3'])
axins.plot(xx,AQ_U*mu_U(xx,a,b),**line_styles_nomarker1['l3'])
axins.plot(xx,AQ_2*mu2(xx),**line_styles_nomarker['l1'])

for l in np.reshape(intervals_raw,-1):
    axins.axvline(l,color='k',lw=.5)

# sub region of the original image
x1, x2, y1, y2 = 1.5,2,0,.1
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticks([x1,x2])
axins.set_yticks([y1,y2])


plt.savefig('imgs/ch4_RM_split_AQ.pdf')
plt.close()

In [11]:
xx = np.linspace(-1,b,1000)

g_σ = lambda x,σ: 1/np.sqrt(2*np.pi*σ**2)*np.exp(-(x/σ)**2/2)

GQ0 = np.zeros_like(xx)
GQ1 = np.zeros_like(xx)
GQ2 = np.zeros_like(xx)
GQ3 = np.zeros_like(xx)

for trial_idx in range(n_trials):
    θ,ω = get_gq(k,α[trial_idx],β[trial_idx])
    for i in range(k):

        GQ0 += ω[i]*g_σ(xx-θ[i],3/k)
        GQ1 += ω[i]*g_σ(xx-θ[i],3/k)
        GQ2 += ω[i]*g_σ(xx-θ[i],8/k)
        GQ3 += ω[i]*g_σ(xx-θ[i],15/k)
    
GQ0 /= n_trials
GQ1 /= n_trials
GQ2 /= n_trials
GQ3 /= n_trials


fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax = fig.add_axes([left, bottom, width, height])

#ax.plot(xx,GQ0,color=colors[4],ls=linestyles[4],lw=.75)
#ax.plot(xx,GQ0,color=colors[4],lw=.2)

ax.plot(xx,GQ1,**line_styles_nomarker['l3'])
ax.plot(xx,GQ1,**line_styles_nomarker1['l3'])

ax.plot(xx,GQ2,**line_styles_nomarker['l2'])
ax.plot(xx,GQ2,**line_styles_nomarker1['l2'])

ax.plot(xx,GQ3,**line_styles_nomarker['l1'])
#ax.plot(xx,GQ3,color=colors[1],lw=.2)

ax.grid(True,linestyle=':',linewidth=.5)

    
for l in np.reshape(intervals_raw,-1):
    ax.axvline(l,color='k',lw=.5)

ax.set_ylim(0,.6)
ax.set_xlim(-1,17)

plt.savefig('imgs/ch4_RM_split_GQ.pdf')
plt.close()